In [2]:
import pandas as pd 
TRAIN_PATH = 'train.csv'

In [3]:
#take 10000 samples
df_sample = pd.read_csv(TRAIN_PATH, nrows=10000)
#Drop key column
df_sample.drop(['key'], axis=1, inplace= True)
df_sample.count()

fare_amount          10000
pickup_datetime      10000
pickup_longitude     10000
pickup_latitude      10000
dropoff_longitude    10000
dropoff_latitude     10000
passenger_count      10000
dtype: int64

In [4]:
#Print all sample where passenger count is 0
df_sample.loc[lambda df:df['passenger_count']==0]
#Remove rows where passenger count is 0
df_sample.drop(df_sample.loc[lambda df:df['passenger_count']==0].index, inplace=True)

In [5]:
#Print all sample where fare amount is negative
df_sample.loc[lambda df:df['fare_amount']<0 ]
#Remove rows where amount is negative
df_sample.drop(df_sample.loc[lambda df:df['fare_amount']<0 ].index, inplace=True)

In [6]:
df_sample.dropna(subset=['pickup_latitude','pickup_longitude',
                 'dropoff_latitude','dropoff_longitude'], how='all', inplace = True)

In [7]:
## Density map of pick-ups and drop-offs

In [8]:
def plot_pickups_dropoffs(dataframe):
    # creating dataframe of shape: 'latitude','longitude','action'
    # where action corresponds to pickup - 'blue', or dropoff - 'red'
    # 'red', 'blue' are then used by 'folium' map to plot dropoff and pickup, respectively
    import os
    import folium
    
    df_length = len(dataframe.index)
    #pickups
    pickups_lon = dataframe['pickup_longitude']
    pickups_lat = dataframe['pickup_latitude']

    blue=[]
    for i in range(df_length):
        blue.append('blue')

    p = {'longitude':pickups_lon, 'latitude':pickups_lat, 'action':blue}
    pickups = pd.DataFrame(data=p)
    #
    #dropoffs
    dropoff_lon = dataframe['dropoff_longitude']
    dropoff_lat = dataframe['dropoff_latitude']
    red=[]
    for i in range(df_length):
        red.append('red')


    d = {'longitude':dropoff_lon, 'latitude':dropoff_lat, 'action':red}
    dropoffs = pd.DataFrame(data=d)
    #
    #building final dataframe (pickups_dropoffs)
    pickups_dropoffs = pd.DataFrame(columns=['longitude', 'latitude', 'action'])
    pickups_dropoffs = pickups_dropoffs.append(pickups, sort=False, ignore_index=True)
    pickups_dropoffs = pickups_dropoffs.append(dropoffs, sort=False, ignore_index=True)
    #
    #building a map
    map_osm = folium.Map(location=[40.73, -73.93], zoom_start=10)
    pickups_dropoffs.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                             radius=0.5, color=row['action'],fill_color=row['action'])
                                             .add_to(map_osm), axis=1)
    #exporting map to html file
    map_osm.save('map.html')
    
    return None

In [9]:
plot_pickups_dropoffs(df_sample)